In [3]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
import random
import glob
import tensorflow as tf
import os as os 
os.chdir('/home/himanshu/cats_dogs/')

In [4]:
# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = '/home/himanshu/cats_dogs/train'
validation_data_dir = '/home/himanshu/cats_dogs/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 20
batch_size = 16


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,             ## Class labels not defined here, to be generated later
        shuffle=False)              ## No random shuffling, so the data from one class gets stacked together
    
    ## Generating bottleneck features for training set
    bottleneck_features_train = model.predict_generator(                    
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'w'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    ## Generating bottleneck features for validation set
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'w'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy'))
    train_labels = np.array(
        [0]*(nb_train_samples/2)+[1]*(nb_train_samples/2))     ## Generating labels for training classes

    validation_data = np.load(open('bottleneck_features_validation.npy'))
    validation_labels = np.array(
        [0]*(nb_validation_samples/2)+[1]*(nb_validation_samples/2))   ## Generating labels for validation set classes

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path) ## Saving trained weights for the model added on top of VGG16 architecture 

## Calling the main initiation functions to save bottleneck features and use further to train the classifier(top_model)
save_bottlebeck_features()
train_top_model()

58900480/58889256 [==============================] - 18s 0us/step
Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Train on 2000 samples, validate on 800 samples
Epoch 1/20
2000/2000 [==============================] - 5s 2ms/step - loss: 0.7226 - acc: 0.7500 - val_loss: 0.2912 - val_acc: 0.8812
Epoch 2/20
2000/2000 [==============================] - 4s 2ms/step - loss: 0.3584 - acc: 0.8585 - val_loss: 0.3276 - val_acc: 0.8688
Epoch 3/20
2000/2000 [==============================] - 4s 2ms/step - loss: 0.3268 - acc: 0.8745 - val_loss: 0.3306 - val_acc: 0.8588
Epoch 4/20
2000/2000 [==============================] - 4s 2ms/step - loss: 0.2542 - acc: 0.8960 - val_loss: 0.2542 - val_acc: 0.9038
Epoch 5/20
2000/2000 [==============================] - 4s 2ms/step - loss: 0.2104 - acc: 0.9225 - val_loss: 0.2768 - val_acc: 0.9062
Epoch 6/20
2000/2000 [==============================] - 4s 2ms/step - loss: 0.1995 - acc: 0.9240 - val_loss: 0.2839 - val_acc: 0.9038
